In [1]:
import sys
from datetime import datetime
import os
import csv
from tqdm import tqdm
import re
from some_func import util

In [2]:
class Grid:
    def __init__(self):
        locating_point = []  # 位于当前网格的定位点
        locating_of_grid = [] # 网格的位置，里面所有点的经纬度平均

In [3]:
def Distance(pointa,pointb):
    distance = 0
    # todo
    return distance

In [4]:
def get_pickup_point(traj_data: dict):
    """

    Args:
        traj_data: 轨迹数据


    Returns:上下车的点的定位点

    """
    pickup_point_list = []
    for key,value in tqdm(traj_data.items()):
        # pickup_point_index = []
        coordinates = value[1]
        status_list = value[2]
        for i in range(1,len(coordinates)):
            if status_list[i] != status_list[i-1]:
                pickup_point_list.append(coordinates[i])

    return pickup_point_list

In [5]:
def load_data():# data:key->taxi_id,value->[[time],[coordinates(not mercator)],[status]],
    data = {}
    lon_min = 180
    lon_max = 0
    lat_min = 90
    lat_max = 0

    file_need_remove = []
    file_need_stay = []
    traj_folder = os.path.join(os.getcwd(), "traj")
    for file_name in tqdm(os.listdir(traj_folder)) :
        if file_name.startswith("traj_") and file_name.endswith(".csv"):
            match = re.search(r"traj_(\d+)\.csv", file_name)
            taxi_id = match.group(1)
            file_path = os.path.join(traj_folder, file_name)
            # data_illegal_count = 0

            time_list = []
            coordinates = []
            status_list = []

            if_use_this_data  = True

            with open(file_path, mode='r', encoding='utf-8') as csv_file:
                csv_reader = csv.DictReader(csv_file)
                for row in csv_reader:
                    ID = int(row['ID'])
                    time_str = row['Time']
                    time = datetime.strptime(time_str, "%H:%M:%S").time()  # 转换为 time 对象
                    longitude = float(row['Longitude'])
                    latitude = float(row['Latitude'])
                    status = int(row['Status'])

                    # 检测到非法数据就弃用该出租车的数据
                    if not (113 < longitude < 114.6):
                        if_use_this_data  = False
                        break
                    if not (22.1 < latitude < 22.7):
                        if_use_this_data  = False
                        break
                    if status != 1 and status != 0 :
                        if_use_this_data  = False
                        break


                    time_list.append(time)
                    coordinates.append([longitude,latitude])
                    status_list.append(status)


                    # lon_min = min(lon_min,longitude)
                    # lon_max = max(lon_max,longitude)
                    # lat_min = min(lat_min,latitude)
                    # lat_max = max(lat_max,latitude)

                    # data.setdefault(ID, [[], [], []])
                    # data[ID][0].append(time)
                    # data[ID][1].append([longitude,latitude])
                    # data[ID][1].append(util.mercator_to_wgs84(longitude, latitude))
                    # data[ID][2].append(status)
                    # print(ID)
                    # print(time,type(time))
                    # print([longitude, latitude])
                    # print(status,type(status))
                    # print("----------------------")

            if if_use_this_data:
                data[taxi_id] = [time_list,coordinates,status_list]
                lon_min = min(lon_min,min(coord[0] for coord in coordinates))
                lon_max = max(lon_max,max(coord[0] for coord in coordinates))
                lat_min = min(lat_min,min(coord[1] for coord in coordinates))
                lat_max = max(lat_max,max(coord[1] for coord in coordinates))
                file_need_stay.append(file_name)
            else:
                file_need_remove.append(file_name)


                # if data_illegal_count > 2:
                #     file_need_remove.append(file_name)
    outputfile = "file_need_stay.txt"
    print(len(file_need_stay))
    with open(outputfile, mode='w', encoding='utf-8') as f:
        for filename in file_need_stay:
            f.write(filename + "\n")  # 每个文件名写入一行
    outputfile = "file_need_remove.txt"
    with open(outputfile, mode='w', encoding='utf-8') as f:
        for filename in file_need_remove:
            f.write(filename + "\n")  # 每个文件名写入一行
    return data,[lon_min,lon_max,lat_min,lat_max]

In [6]:
# 参数

# lon经度 lat纬度


k = 20# 网格的大小，单位为米
lameda = 0# 网格密度阈值
fai = 1.3 # 定义9里面那个参数

In [ ]:
traj_data,district = load_data()
print(district)


 65%|██████▍   | 5820/8974 [02:54<01:04, 48.98it/s]

In [ ]:
# # 这段是拿来把原本的traj文件夹里面的数据分割成两部分的，不用管
#
#
#
#
# import os
# import shutil
#
# # 定义路径
# current_dir = os.getcwd()  # 获取当前目录
# traj_folder = os.path.join(current_dir, "traj")
# traj_remove_folder = os.path.join(current_dir, "traj_remove")
# file_need_remove = os.path.join(current_dir, "file_need_remove.txt")
#
# # 创建 traj_remove 文件夹（如果不存在）
# if not os.path.exists(traj_remove_folder):
#     os.makedirs(traj_remove_folder)
#
# # 读取 file_need_remove.txt
# if os.path.exists(file_need_remove):
#     with open(file_need_remove, mode="r", encoding="utf-8") as f:
#         filenames = [line.strip() for line in f.readlines() if line.strip()]
# else:
#     print(f"文件 {file_need_remove} 不存在！")
#     exit(1)
#
# # 移动文件
# for filename in filenames:
#     source_path = os.path.join(traj_folder, filename)
#     target_path = os.path.join(traj_remove_folder, filename)
#
#     if os.path.exists(source_path):
#         shutil.move(source_path, target_path)  # 移动文件
#         print(f"已移动: {filename}")
#     else:
#         print(f"文件 {filename} 不存在于 {traj_folder} 中！")
#
# print("处理完成。")

记录，第一次
```python
if not (113 < longitude < 114.6):
    if_use_this_data  = False
    break
if not (22.1 < latitude < 22.7):
    if_use_this_data  = False
    break
```
8975条轨迹被保留
5753条轨迹被移除
[113.310249, 114.567619, 22.157217, 22.699966]

记录，第二次
```python
if not (113.4 < longitude < 114.4):
    if_use_this_data  = False
    break
if not (22.15 < latitude < 22.6):
    if_use_this_data  = False
    break
```
184条轨迹被保留
14544条轨迹被移除


In [ ]:
count = 0
for key,value in traj_data.items():
    count += len(value[1])
print(count)

In [ ]:
pickup_points = get_pickup_point(traj_data)
print(len(pickup_points))

In [ ]:
# 这里可以做个上下车点可视化



In [ ]:
# 才发现网格的lon_max啥的是上下车点的lon_max,
lon_min = min([pickup_point[0] for pickup_point in pickup_points])
lon_max = max([pickup_point[0] for pickup_point in pickup_points])
lat_min = min([pickup_point[1] for pickup_point in pickup_points])
lat_max = max([pickup_point[1] for pickup_point in pickup_points])
print(lon_min,lon_max,lat_min,lat_max)

In [ ]:
# 创建网格，将网格四个点映射为mercator坐标，获得四个点坐标，右经度-左经度获得长度，长度/k,向下取整，得到网格的横的数量
# 同理得到网格竖的数量，创建grids[]，将点映射到网格时，以左下角为准，比如说一个点属于网格[i][j],以左下角为准





# 遍历上下车点，将其映射到grid，添加进grid

# 计算每个grid的点数量，超过lameda就计算平均位置，记录index